In [ ]:
# generate cellpose dataset
from spacr.io import prepare_cellpose_dataset

input_root = 'path'

prepare_cellpose_dataset(input_root, augment_data=True, train_fraction=0.8, n_jobs=None)

In [ ]:
# train cellpose model
from spacr.submodules import train_cellpose
%matplotlib inline

settings = {'src':'path',
            'test':False,
            'normalize':False,
            'percentiles':None,
            'invert':False,
            'grayscale':True,
            'rescale':False,
            'circular':False,
            'channels':[0,0],
            'model_name':'test',
            'model_type':'cyto',
            'Signal_to_noise':10,
            'background':200,
            'remove_background':False,
            'learning_rate':0.2,
            'weight_decay':1e-05,
            'batch_size':8,
            'n_epochs':25000,
            'from_scratch':False,
            'diameter':30,
            'resize':False,
            'target_dimensions':1000,
            'verbose':True}

train_cellpose(settings)